In [1]:
import collections
import shutil
import tempfile
import zipfile
from pathlib import Path
from typing import Tuple
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from anomalib.config import get_configurable_parameters
from anomalib.data import get_datamodule
from anomalib.data.inference import InferenceDataset
from anomalib.models import get_model
from anomalib.models.patchcore.lightning_model import PatchcoreLightning
from anomalib.post_processing.post_process import compute_mask
from anomalib.utils.callbacks import get_callbacks
from anomalib.utils.loggers import get_experiment_logger
from pytorch_lightning import Trainer, seed_everything
from skimage.morphology import dilation
from skimage.segmentation import find_boundaries
from torch.utils.data import DataLoader
     


/work/scratch/tyang/miniconda3/env/anomalib_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
OpenVINO is not installed. Please install OpenVINO to use OpenVINOInferencer.
OpenVINO is not installed. Please install OpenVINO to use OpenVINOInferencer.


To use wandb logger install it using `pip install wandb`


In [2]:
DATA_FOLDER = Path("/home/students/tyang/Downloads")
IDRID_ZIP_LOCATION = DATA_FOLDER / "B. Disease Grading.zip"

PATCHCORE_CONFIG_PATH = "/home/students/tyang/anomalib/src/anomalib/models/patchcore/idrid.yaml"


In [ ]:
with tempfile.TemporaryDirectory() as tmp_data_dir:
    with zipfile.ZipFile(IDRID_ZIP_LOCATION, "r") as idrid_zip_ref:
        idrid_zip_ref.extractall(tmp_data_dir)

        tmp_data_dir = Path(tmp_data_dir)
        gt_path = tmp_data_dir / "B. Disease Grading" / "2. Groundtruths"
        original_img_path = tmp_data_dir / "B. Disease Grading" / "1. Original Images"

        normal_img_dst = DATA_FOLDER / "idrid" / "normal"
        normal_img_dst.mkdir(exist_ok=True, parents=True)

        dr_img_dst = DATA_FOLDER / "idrid" / "dr"
        dr_img_dst.mkdir(exist_ok=True, parents=True)

        for csv_fn, dir_name in [
            ("a. IDRiD_Disease Grading_Training Labels.csv", "a. Training Set"),
            ("b. IDRiD_Disease Grading_Testing Labels.csv", "b. Testing Set"),
        ]:
            gt_csv = pd.read_csv(gt_path / csv_fn)

            no_retinopathy = gt_csv[gt_csv["Retinopathy grade"] == 0]
            for _, no_retinopathy_img in no_retinopathy.iterrows():
                shutil.copy(
                    original_img_path
                    / dir_name
                    / (no_retinopathy_img["Image name"] + ".jpg"),
                    normal_img_dst
                    / f"{dir_name.strip()}_{no_retinopathy_img['Image name']}.jpg",
                )

            for retinopathy_grade in range(1, 5):
                cur_dst = dr_img_dst / str(retinopathy_grade)
                cur_dst.mkdir(exist_ok=True, parents=True)

                retinopathy_subset = gt_csv[
                    gt_csv["Retinopathy grade"] == retinopathy_grade
                ]
                for _, retinopathy_img in retinopathy_subset.iterrows():
                    shutil.copy(
                        original_img_path
                        / dir_name
                        / (retinopathy_img["Image name"] + ".jpg"),
                        cur_dst
                        / f"{dir_name.strip()}_{retinopathy_img['Image name']}.jpg",
                    )

In [3]:


config = get_configurable_parameters(
    model_name="patchcore", config_path=PATCHCORE_CONFIG_PATH
)
if config.project.seed:
    seed_everything(config.project.seed)

datamodule = get_datamodule(config)
model = get_model(config)
experiment_logger = get_experiment_logger(config)
callbacks = get_callbacks(config)
     

trainer = Trainer(**config.trainer, logger=experiment_logger, callbacks=callbacks)
trainer.fit(model=model, datamodule=datamodule)
     


/home/students/tyang/anomalib/src/anomalib/config/config.py:243: UserWarning: The seed value is now fixed to 0. Up to v0.3.7, the seed was not fixed when the seed value was set to 0. If you want to use the random seed, please select `None` for the seed value (`null` in the YAML file) or remove the `seed` key from the YAML file.
  warn(
/home/students/tyang/anomalib/src/anomalib/config/config.py:280: UserWarning: config.project.unique_dir is set to False. This does not ensure that your results will be written in an empty directory and you may overwrite files.
  warn(
/work/scratch/tyang/miniconda3/env/anomalib_env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Loading model wide_resnet50_2 from torchvision


/work/scratch/tyang/miniconda3/env/anomalib_env/lib/python3.10/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/work/scratch/tyang/miniconda3/env/anomalib_env/lib/python3.10/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/work/scratch/tyang/miniconda3/env/anomalib_env/lib/python3.10/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/work/scratch/tyang/miniconda3/env/anomalib_env/lib/python3.10/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,
/home/students/tyang/anomalib/src/anomalib/utils/callbacks/__init__.py:153: UserWarning: Export option: None not found. Defaulting to no model export
  warnings.warn(f"Export opt

Epoch 0:  50%|█████     | 3/6 [00:08<00:08,  2.98s/it, loss=nan]

/work/scratch/tyang/miniconda3/env/anomalib_env/lib/python3.10/site-packages/pytorch_lightning/loops/optimization/optimizer_loop.py:138: UserWarning: `training_step` returned `None`. If this was on purpose, ignore this warning...
  self.warning_cache.warn("`training_step` returned `None`. If this was on purpose, ignore this warning...")


Epoch 0:  83%|████████▎ | 5/6 [00:09<00:01,  1.84s/it, loss=nan]








































































































































































































































































































































































































Selecting Coreset Indices.: 100%|██████████| 13824/13824 [00:40<00:00, 344.44it/s]


Epoch 0: 100%|██████████| 6/6 [00:55<00:00,  9.21s/it, loss=nan, pixel_F1Score=0.109, pixel_AUROC=0.807]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 6/6 [01:13<00:00, 12.17s/it, loss=nan, pixel_F1Score=0.109, pixel_AUROC=0.807]


In [4]:
def prepare_predictor(
    model_config_path: str, model_weight_path: str
) -> Tuple[PatchcoreLightning, Trainer]:
    """Load anomalib model"""
    pred_config = get_configurable_parameters(config_path=model_config_path)
    pred_config.trainer.resume_from_checkpoint = model_weight_path
    pred_config.visualization.show_images = False
    pred_config.visualization.save_images = False

    model = get_model(pred_config)
    pred_callbacks = get_callbacks(pred_config)

    pred_trainer = Trainer(callbacks=pred_callbacks, **pred_config.trainer)
    return model, pred_trainer


def get_prediction(
    img_path: str,
    model: PatchcoreLightning,
    pred_trainer: Trainer,
    img_size: Tuple[int, int] = (256, 256),
):
    """Get a prediction for an image at img_path by a model with given config"""

    dataset = InferenceDataset(
        img_path,
        image_size=img_size,
        transform=None,
    )
    dataloader = DataLoader(dataset)
    return pred_trainer.predict(model=model, dataloaders=[dataloader])


def add_segmentation_boundary(
    image: np.ndarray,
    anomaly_map: np.ndarray,
    threshold: float = 0.5,
    thickness: int = 20,
    color: Tuple[int, int, int] = (255, 0, 0),
):
    """Add a segmentation boundary based on an anomaly map"""
    marked = np.copy(image)
    anomaly_mask = compute_mask(anomaly_map, threshold)
    boundaries = find_boundaries(anomaly_mask)
    outlines = dilation(boundaries, np.ones((thickness, thickness)))
    marked[outlines] = color
    return marked

In [ ]:

model_weight_path = (
    "/home/students/tyang/anomalib/results/patchcore/idrid/run/weights/lightning/model.ckpt"

)
model, pred_trainer = prepare_predictor(PATCHCORE_CONFIG_PATH, model_weight_path)

patchcore_results = collections.defaultdict(dict)
for grade in range(1, 5):
    img_folder = (
        DATA_FOLDER / "idrid" / "dr" / str(grade)
        if grade > 0
        else DATA_FOLDER / "idrid" / "normal"
    )
    img_paths = sorted((img_folder).glob("*.jpg"))[:4]
    for img_path in img_paths:
        patchcore_results[grade][img_path] = get_prediction(
            img_path, model, pred_trainer
        )

In [ ]:
for grade in range(1, 5):
    grade_results = patchcore_results[grade]

    plt.figure(figsize=(10, 15))
    for img_idx, img_fn in enumerate(grade_results):
        img = cv2.imread(img_fn.as_posix())[:, :, ::-1]  # BGR RGB conversion

        anomaly_map = np.squeeze(grade_results[img_fn][0]["anomaly_maps"].numpy())
        anomaly_map_original_size = cv2.resize(
            anomaly_map, (img.shape[1], img.shape[0])
        )
        segmentation_boundary = add_segmentation_boundary(
            img, anomaly_map_original_size, threshold=0.3
        )

        plt.subplot(len(grade_results), 3, img_idx * 3 + 1)
        plt.imshow(img)
        plt.axis("off")

        plt.subplot(len(grade_results), 3, img_idx * 3 + 2)
        plt.imshow(img)
        plt.imshow(anomaly_map_original_size, alpha=0.5)
        plt.axis("off")

        plt.subplot(len(grade_results), 3, img_idx * 3 + 3)
        plt.imshow(segmentation_boundary)
        plt.axis("off")

    plt.subplots_adjust(wspace=0.05, hspace=-0.7)
    plt.suptitle(f"PatchCore results for DR grade {grade}", y=0.75)
    plt.show()
     